In [6]:
import pygame
import random
from pygame import mixer
import json
import os

colors = [
    (0, 0, 0),
    (120, 37, 179),
    (100, 179, 179),
    (80, 34, 22),
    (80, 134, 22),
    (180, 34, 22),
    (180, 34, 122),
]

class Figure:
    x = 0
    y = 0

    figures = [
        [[1, 5, 9, 13], [4, 5, 6, 7]],  # Hình I
        [[4, 5, 9, 10], [2, 6, 5, 9]],  # Hình Z
        [[6, 7, 9, 10], [1, 5, 6, 10]], # Hình Z ngược
        [[1, 2, 5, 9], [0, 4, 5, 6], [1, 5, 9, 8], [4, 5, 6, 10]], # Hình L
        [[1, 2, 6, 10], [5, 6, 7, 9], [2, 6, 10, 11], [3, 5, 6, 7]], # Hình L ngược
        [[1, 4, 5, 6], [1, 4, 5, 9], [4, 5, 6, 9], [1, 5, 6, 9]], # Hình T
        [[1, 2, 5, 6]], # Hình O
    ]
#Khi một Figure được khởi tạo, vị trí của nó được xác định bởi tham số x và y.
#self.type là loại hình khối, được chọn ngẫu nhiên từ danh sách figures.
#self.color là màu của khối, cũng được chọn ngẫu nhiên từ danh sách colors.
#self.rotation là trạng thái xoay của khối, khởi đầu là 0 (không xoay).
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.type = random.randint(0, len(self.figures) - 1)
        self.color = random.randint(1, len(colors) - 1)
        self.rotation = 0

    def image(self):#Phương thức này trả về hình ảnh hiện tại của khối (dạng xoay hiện tại của khối).
        return self.figures[self.type][self.rotation]

    def rotate(self):
        #Method này thực hiện việc xoay khối. Nó cập nhật trạng thái xoay bằng cách tăng chỉ số rotation và 
        #dùng phép chia lấy dư để đảm bảo giá trị xoay nằm trong giới hạn của số trạng thái có thể xoay của khối.
        self.rotation = (self.rotation + 1) % len(self.figures[self.type])


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
class Tetris:#Đây là lớp chính của trò chơi, quản lý logic của trò chơi Tetris, như tạo khối mới, di chuyển khối, kiểm tra va chạm, và xử lý các dòng đầy.
    def __init__(self, height, width):
        self.level = 2
        self.x = 100
        self.y = 60
        self.zoom = 20
        self.figure = None
    
        self.height = height
        self.width = width
        self.field = []
        self.score = 0
        self.state = "start"
        for i in range(height):
            new_line = []
            for j in range(width):
                new_line.append(0)
            self.field.append(new_line)

    # def show_field(self):  # Add 'self' parameter
    #     for row in self.field:
    #         print(" ".join(map(str, row)))

    def new_figure(self):
        self.figure = Figure(3, 0)
        
    # def show_figure(self):
    #     return self.figure.image()

    def intersects(self):
        intersection = False
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.figure.image():
                    if i + self.figure.y > self.height - 1 or \
                            j + self.figure.x > self.width - 1 or \
                            j + self.figure.x < 0 or \
                            self.field[i + self.figure.y][j + self.figure.x] > 0:
                        intersection = True
        return intersection

    def break_lines(self):
        lines = 0
        for i in range(1, self.height):
            zeros = 0
            for j in range(self.width):
                if self.field[i][j] == 0:
                    zeros += 1
            if zeros == 0:
                lines += 1
                for i1 in range(i, 1, -1):
                    for j in range(self.width):
                        self.field[i1][j] = self.field[i1 - 1][j]
        self.score += lines ** 2

    def go_space(self):
        while not self.intersects():
            self.figure.y += 1
        self.figure.y -= 1
        self.freeze()

    def go_down(self):
        self.figure.y += 1
        if self.intersects():
            self.figure.y -= 1
            self.freeze()

    def freeze(self):
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.figure.image():
                    self.field[i + self.figure.y][j + self.figure.x] = self.figure.color
        self.break_lines()
        self.new_figure()
        if self.intersects():
            self.save_scores()
            self.state = "gameover"

    def go_side(self, dx):
        old_x = self.figure.x
        self.figure.x += dx
        if self.intersects():
            self.figure.x = old_x

    def rotate(self):
        old_rotation = self.figure.rotation
        self.figure.rotate()
        if self.intersects():
            self.figure.rotation = old_rotation

    def figure_speed():
        pass

    def save_scores(self):
            scores = self.load_scores()        # Đọc điểm số hiện tại từ file
            scores.append(self.score)        # Thêm điểm số mới vào danh sách
            scores = sorted(scores, reverse=True)[:10]        # Sắp xếp danh sách và giữ lại top 10
            with open("tetris_scores2.json", "w") as file:        # Ghi lại điểm số vào file
                json.dump(scores, file)

    def load_scores(self):
        if os.path.exists("tetris_scores2.json"):        # Tải điểm số từ file, nếu không có thì trả về danh sách trống
            with open("tetris_scores2.json", "r") as file:
                return json.load(file)
        return []


In [8]:
def ve_man_hinh_chinh():
    pass
def ve_nut_nhap_ten_player():
    pass
def draw_gameover():
    pass
def draw_pause_screen():
    pass
def sound():
    pass

pressing_down = False
pressing_left = False
pressing_right = False

# Biến để theo dõi thời gian giữ phím
hold_time = 0
initial_move_time = 10  # Thời gian để di chuyển chậm (trong số vòng lặp)
fast_move_time = 5      # Thời gian để di chuyển nhanh (trong số vòng lặp)
game = Tetris(height=20, width=10)  
  
def display_game(size, state, fps, counter, pressing_down): #hàm này để hiển thị game

    pygame.init()#pygame.init(): This is a function provided by the pygame module to initialize all the imported pygame modules (like display, mixer, etc.)
    clock = pygame.time.Clock() #tạo đối tượng để quản lý thời gian trong game
    screen = pygame.display.set_mode(size, pygame.RESIZABLE) #tạo cửa sổ game với kích thước 400x500 pixel
    pygame.display.set_caption("Tetris")    #đặt tên cho game
    Icon = pygame.image.load(r"D:\PythonCode\Python_Assignment\Python_project\APT_picture\APT_icon.webp")   #đường dẫn ảnh icon
    pygame.display.set_icon(Icon)   #thiết lập icon cho game

    mixer.init()     # Starting the mixer 
    mixer.music.load(r"D:\PythonCode\Python_Assignment\Python_project\apt---rosé--bruno-mars--lyrics-video.mp3")     # Loading the song 
    mixer.music.set_volume(0.7)     # Setting the volume 
    mixer.music.play()      # Start playing the song 
    
    # initial_speed = 30  # Tốc độ rơi ban đầu
    # decay_factor = 0.9  # Hệ số giảm tốc độ rơi mỗi lần lên level (10%)
    while not state: #vòng lặp chạy cho đến khi state = True
        if not mixer.music.get_busy():
            mixer.music.play()  # Kiểm tra nếu nhạc đã dừng để phát lại từ đầu

        if game.figure is None:
            game.new_figure()   #khi
            
        counter += 1
        if counter > 100000:
            counter = 0
        if counter % (fps // game.level // 2) == 0 or pressing_down:
            if game.state == "start":
                game.go_down() #khi game bắt đầu thì khối tự rơi xuống
    #     fall_speed = int(initial_speed * (decay_factor ** (game.level - 1)))

    # # Kiểm tra điều kiện rơi
    #     if counter % fall_speed == 0 or pressing_down:
    #         if game.state == "start":
    #             game.go_down()

    #Nếu người dùng thao tác với bàn phím hoặc chuột thì sẽ có các sự kiện được tạo ra và lưu trong pygame.event.get()
        for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    done = True
                    
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        game.rotate()
                    if event.key == pygame.K_DOWN:
                        pressing_down = True
                    if event.key == pygame.K_LEFT:
                        pressing_left = True
                        hold_time = 0  # Reset hold time khi bắt đầu giữ phím
                    if event.key == pygame.K_RIGHT:
                        pressing_right = True
                        hold_time = 0  # Reset hold time khi bắt đầu giữ phím
                    if event.key == pygame.K_SPACE:
                        game.go_space()
                    if event.key == pygame.K_ESCAPE:
                        game.__init__(20, 10)

                if event.type == pygame.KEYUP:
                    if event.key == pygame.K_DOWN:
                        pressing_down = False
                    if event.key == pygame.K_LEFT:
                        pressing_left = False
                    if event.key == pygame.K_RIGHT:
                        pressing_right = False
                        hold_time = 0  # Reset hold time khi thả phím

        # Di chuyển liên tục khi các phím được giữ
        if pressing_left or pressing_right:
            hold_time += 1  # Tăng hold_time khi phím vẫn được giữ

            # Sử dụng tốc độ khác nhau dựa trên hold_time
            if hold_time < initial_move_time:
                if pressing_left and counter % (fps // game.level // 4) == 0:  # Di chuyển chậm
                    game.go_side(-1)
                if pressing_right and counter % (fps // game.level // 4) == 0:  # Di chuyển chậm
                    game.go_side(1)
            else:
                if pressing_left and counter % (fps // game.level // 8) == 0:  # Di chuyển nhanh
                    game.go_side(-1)
                if pressing_right and counter % (fps // game.level // 8) == 0:  # Di chuyển nhanh
                    game.go_side(1)

    #Create a background image
        background_image = pygame.image.load(r"D:\PythonCode\Python_Assignment\Python_project\APT_picture\APT3.jpg")
        background_image = pygame.transform.scale(background_image, (400, 500))
        screen.blit(background_image, [0, 0])

        for i in range(game.height):
            for j in range(game.width):
                pygame.draw.rect(screen, (128, 128, 128), [game.x + game.zoom * j, game.y + game.zoom * i, game.zoom, game.zoom], 1)
                if game.field[i][j] > 0:
                    pygame.draw.rect(screen, colors[game.field[i][j]],
                                    [game.x + game.zoom * j + 1, game.y + game.zoom * i + 1, game.zoom - 2, game.zoom - 1])

        if game.figure is not None:
            for i in range(4):
                for j in range(4):
                    p = i * 4 + j
                    if p in game.figure.image():
                        pygame.draw.rect(screen, colors[game.figure.color],
                                        [game.x + game.zoom * (j + game.figure.x) + 1,
                                        game.y + game.zoom * (i + game.figure.y) + 1,
                                        game.zoom - 2, game.zoom - 2])


        font = pygame.font.SysFont('Calibri', 25, True, False)
        font1 = pygame.font.SysFont('Calibri', 65, True, False)
        text = font.render("Score: " + str(game.score), True, (0, 0, 0))
        text_game_over = font1.render("Game Over", True, (255, 125, 0))
        text_game_over1 = font1.render("Press ESC", True, (255, 215, 0))

        screen.blit(text, [0, 0])
        if game.state == "gameover":
            screen.blit(text_game_over, [20, 200])
            screen.blit(text_game_over1, [25, 265])

        pygame.display.flip()
        clock.tick(fps)

    pygame.quit()



In [9]:
input_size = (400, 500)   #kích thước cửa sổ game (400x500) pixel
input_state = False    #biến done để kiểm tra xem game đã kết thúc chưa
input_fps = 25    #thiết lập tốc độ khung hình 
input_counter = 0 #biến đếm quản lý thời gian và tốc độ rơi của khối (counter càng lớn thì khối rơi càng nhanh)
input_pressing_down = False   #biến kiểm tra xem phím mũi tên xuống có được nhấn không


In [ ]:
display_game(input_size, input_state, input_fps, input_counter, input_pressing_down) #gọi hàm display_game để hiển thị game

UnboundLocalError: cannot access local variable 'pressing_left' where it is not associated with a value

: 

In [6]:
game.load_scores()  # Load điểm số từ file

[0]